In [1]:
import numpy as np
import tifffile as tif
import os

# data manager and analysis
import vodex as vx
import numan as nu
import pandas as pd

Matplotlib created a temporary config/cache directory at /tmp/matplotlib-lajqxkq_ because the default path (/home/ply/.config/matplotlib) is not a writable directory; it is highly recommended to set the MPLCONFIGDIR environment variable to a writable directory, in particular to speed up the import of Matplotlib and to better support multiprocessing.


Get the project directory:

In [2]:
# please provedi FULL, not relative, path to the folder 
project_folder = "/home/ply/repos/numan_dev/numan/data/hz09"
project = nu.Project(project_folder)

project.check_exists("processed/drift_corrected_af_to1000")

We will also set the processed directory as our working directory, this step is important , since all the paths later are relative to this folder. Verify that the output of the cell is the "processed" folder inside your project folder. 

In [3]:
project.activate("processed")
os.getcwd()

'/home/ply/repos/numan_dev/numan/data/hz09/processed'

# Load experiment: 

* Use drift corrected data

In [4]:
experiment = vx.Experiment.load("experiment_truncated_drift_corrected.db")

array([ 6,  7,  8,  9, 10, 11, 12, 13, 14])

# Create dff movie

In [6]:
project.create("processed/dff_movie")

In [7]:
# batch size 25 takes about 30 Gb of RAM to process
batch_size = 198 # in volumes, needs to be divisible by step_size so it's 198 and not 200
step_size = 9 # in volumes
baseline_volumes = [0,1,2]
spacing_xyz = [1.62, 1.62, 3.83] # in um

nu.Preprocess(experiment).calculate_dff("dff_movie", batch_size, step_size, baseline_volumes,
                      resolution_xyz = spacing_xyz, blur_sigma=1, 
                      verbose=False, save_baseline=True)

/home/ply/miniconda3/envs/numan_dev/lib/python3.10/site-packages/vodex/experiment.py:601: UserWarning:

list_volumes will be removed in vodex 1.1.0 use volumes property instead.

 91%|█████████ | 10/11 [07:15<00:43, 43.52s/it]


### Now cerate an experiment that uses dff_movie as the data
Before we make a new experiment, let's close the databse connection of the existing one.

In [ ]:
# update files info to use the new removed_offset folder instead of the original raw data
data_dir = "dff_movie"

# volumes info is the same as in the original experiment
frames_per_volume = experiment.frames_per_volume
starting_slice = experiment.starting_slice

# close the old experiment
experiment.close()
# create new experiment
experiment = vx.Experiment.from_dir(data_dir, frames_per_volume, starting_slice, verbose=True)
# add time annotation 
stimuli_volumes_df = pd.read_csv("stimuli_truncated_timelines.csv")
experiment.add_annotations_from_volume_annotation_df(stimuli_volumes_df)

# for now there will be no description of the labels when adding them from the volume annotation df
# so you will see description : None for all labels in the labels_df
# will add this option in later versions of vodex
experiment.labels_df

**Make sure everything except the filenames looks identical to the experiment we defined at the beginning** and then , if it does , save it for future use: 

In [16]:
experiment.save("experiment_dff.db")
experiment.close()

Copied 2643 of 2643 pages...
